# Simulator

In [1]:
import pandas as pd

import payment_simulator as ps
from payment_simulator.anomaly import AnomalyGenerator
from payment_simulator.networks import SimplePaymentNetwork
from payment_simulator.utils import random_payment_timing, random_payment_value

In [2]:
TOTAL_BANKS = 20
INITIAL_BANKS = 3
INCREMENT_BANKS = 5
ALPHA = 0.00001
AVG_PAYMENTS = 35
ALLOW_SELF_LOOP = False

In [3]:
sim_periods = list(range(15))

sim_params = {
    "open_time": "06:30:00",
    "close_time": "18:30:00",
    "value_fn": random_payment_value,
    "timing_fn": random_payment_timing,
}

payment_network = SimplePaymentNetwork(
    total_banks=TOTAL_BANKS,
    avg_payments=AVG_PAYMENTS,
    alpha=ALPHA,
    allow_self_loop=ALLOW_SELF_LOOP
)

anomaly_generator = AnomalyGenerator(
    anomaly_start=5,
    anomaly_end=13,
    prob_start=0.8,
    prob_end=1,
    lambda_start=5.5,
    lambda_end=12.5,
    rate=0.5,
)

In [4]:
normal_transactions = ps.TransactionSim(sim_id=1, network=payment_network, **sim_params)
normal_transactions.run(sim_periods)

payments1 = normal_transactions.get_payments_df()

pd.concat([payments1.head(3), payments1.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,17:54:40,0,8,1,1.406291
1,0,15:49:51,0,8,1,0.360033
2,0,14:47:44,0,8,1,1.753326
10497,14,07:43:37,18,2,1,0.186491
10498,14,09:48:00,19,9,1,1.396396
10499,14,08:12:19,19,14,1,1.864318


In [5]:
anomaly_transactions = ps.TransactionSim(sim_id=2, network=payment_network, anomaly_gen=anomaly_generator, **sim_params)
anomaly_transactions.run(sim_periods, anomalous_bank = [1,3,5])

payments2 = anomaly_transactions.get_payments_df()

pd.concat([payments2.head(3), payments2.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,13:39:52,0,6,1,0.351316
1,0,12:35:48,0,6,1,7.133166
2,0,17:28:44,0,6,1,0.459810
10497,14,09:44:12,18,17,1,3.222481
10498,14,17:40:34,18,12,1,2.550830
10499,14,11:43:35,19,11,1,0.292492


In [6]:
print(f"Total Value of Normal RTGS  : {payments1['Value'].sum():.3f} from {payments1.shape[0]} transactions")
print(f"Total Value of Anomaly RTGS : {payments2['Value'].sum():.3f} from {payments1.shape[0]} transactions")

Total Value of Normal RTGS  : 17333.730 from 10500 transactions
Total Value of Anomaly RTGS : 32159.608 from 10500 transactions
